In [5]:
# Core imports (safety cell)
# This cell ensures common names like `np`, `pd`, `plt`, `sns` are defined
# so running individual later cells or static analysis won't report NameError.
try:
    import numpy as np
except Exception:
    np = None
try:
    import pandas as pd
except Exception:
    pd = None
try:
    import matplotlib.pyplot as plt
except Exception:
    plt = None
try:
    import seaborn as sns
except Exception:
    sns = None


# EV Charging — Dataset readiness, EDA and prototype models

This notebook loads the residential EV charging dataset (local copy if available), runs exploratory data analysis (EDA), implements simple rule-of-thumb checks to decide if there is enough data to train a neural network, builds a small baseline model, and trains a tiny PyTorch MLP as a feasibility test.

Goals:
- Inspect dataset size, span and per-user distribution.
- Provide automated heuristics for 
 for a small NN.
- Implement a linear baseline and a minimal PyTorch MLP.
- Summarize findings and next steps for the class project.

## 1) Imports and environment checks

In [4]:
# Standard data stack
import os
import math
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

# ML libs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, r2_score

# Try to import torch (optional)
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    has_torch = True
    device = torch.device("cpu")
    try:
        if torch.backends.mps.is_available():
            device = torch.device("mps")
        elif torch.cuda.is_available():
            device = torch.device("cuda")
    except Exception:
        # some torch builds may not expose mps attribute cleanly
        device = torch.device("cpu")
except Exception as e:
    has_torch = False
    device = None

print('has_torch =', has_torch, 'device =', device)

has_torch = True device = mps


In [1]:
import os
from pathlib import Path
import sys

# Ensure notebook runs from the repository root so relative paths work
cwd = Path.cwd()
repo_root = None
markers = ['README.md', '.git', 'pyproject.toml']

# 1) Search upwards from cwd
for parent in [cwd] + list(cwd.parents):
    if any((parent / m).exists() for m in markers):
        repo_root = parent
        break

# 2) If not found, search immediate child directories (this handles kernels started one level above the repo)
if repo_root is None:
    for child in cwd.iterdir():
        try:
            if child.is_dir() and any((child / m).exists() for m in markers):
                repo_root = child
                break
        except PermissionError:
            continue

# 3) As a last resort, do a shallow two-level search (child's children)
if repo_root is None:
    for child in cwd.iterdir():
        try:
            if not child.is_dir():
                continue
            for g in child.iterdir():
                try:
                    if g.is_dir() and any((g / m).exists() for m in markers):
                        repo_root = g
                        break
                except PermissionError:
                    continue
            if repo_root is not None:
                break
        except PermissionError:
            continue

# If still not found, default to cwd
if repo_root is None:
    repo_root = cwd

# Change cwd if necessary
if repo_root != cwd:
    try:
        os.chdir(repo_root)
        print('Changed cwd to repo root:', Path.cwd())
    except Exception as e:
        # don't crash the notebook; print diagnostic and continue from current cwd
        print('Failed to change cwd to repo_root:', repo_root, 'error:', e)
else:
    print('Repo root appears to be current working directory:', repo_root)

# Optionally add repo root to sys.path so notebook imports can find project modules
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))
    print('Inserted repo_root to sys.path')


Changed cwd to repo root: /Users/cyrils/Developer/Python/NeuralNetworks
Inserted repo_root to sys.path


## 2) Load dataset (tries a few local paths)

In [2]:
# Robust dataset loader: try DATASET_PATH env var first, then project-relative candidates.
from pathlib import Path
import io
import pandas as pd
import re

candidates = [
    Path('data/trondheim/Dataset_2_Hourly_EV_per_user.csv'),
    Path('data/trondheim/Dataset_2_Hourly_EV_per_user_sample.csv'),
]

found = None
# prefer explicit env var
dp = os.getenv('DATASET_PATH')
if dp:
    p = Path(dp)
    if p.exists():
        found = p

if found is None:
    for p in candidates:
        if p.exists():
            found = p
            break
    # glob fallback
    if found is None:
        import glob
        matches = list(Path('data/trondheim').glob('*.csv')) if Path('data/trondheim').exists() else []
        if matches:
            found = matches[0]


def robust_read_ev(path: Path) -> pd.DataFrame:
    text = path.read_text(encoding='utf-8', errors='replace')
    # detect delimiter from header
    first_line = text.splitlines()[0]
    if ';' in first_line:
        sep=';'
    elif '\t' in first_line:
        sep='\t'
    else:
        sep=','

    # First-pass read with detected separator using python engine (more permissive)
    try:
        df = pd.read_csv(io.StringIO(text), sep=sep, engine='python')
    except Exception:
        df = pd.read_csv(io.StringIO(text), sep=sep, engine='python', on_bad_lines='skip')

    # If we ended up with a single column that still contains separators, reparse explicitly
    if df.shape[1] == 1:
        sample = df.iloc[0,0] if len(df) else ''
        if isinstance(sample, str) and (';' in sample or '\t' in sample):
            df = pd.read_csv(io.StringIO(text), sep=';', engine='python', on_bad_lines='skip')

    # Normalize column names
    df.columns = [c.strip() if isinstance(c, str) else c for c in df.columns]

    # Clean numeric-looking columns: remove tabs, non-breaking spaces, stray spaces, convert comma decimals
    def clean_numeric_series(s: pd.Series) -> pd.Series:
        s = s.fillna('').astype(str)
        s = s.str.replace('\t', '', regex=True)
        s = s.str.replace('\xa0', '', regex=False)
        s = s.str.replace(' ', '', regex=False)
        s = s.str.replace('\u00A0', '', regex=False)
        s = s.str.replace(',', '.', regex=False)
        # remove any stray semicolons or letters like 'NA'
        s = s.str.replace(';', '', regex=False)
        s = s.replace({'NA': '', 'na': '', 'NaN': ''})
        return pd.to_numeric(s, errors='coerce')

    # Heuristic: energy columns contain words like 'Synthetic' or 'Flex' or 'kW' or 'kwh'
    energy_cols = [c for c in df.columns if isinstance(c, str) and (re.search(r'Synthetic|Flex|kW|kwh', c, re.I))]
    for c in energy_cols:
        df[c] = clean_numeric_series(df[c])

    # parse date_from/date_to if present
    if 'date_from' in df.columns:
        df['date_from'] = pd.to_datetime(df['date_from'].astype(str).str.strip(), dayfirst=True, errors='coerce', format='%d.%m.%Y %H:%M')
    if 'date_to' in df.columns:
        df['date_to'] = pd.to_datetime(df['date_to'].astype(str).str.strip(), dayfirst=True, errors='coerce', format='%d.%m.%Y %H:%M')

    return df

# Load
if found is None:
    print('No local file found. Please download the Kaggle dataset and place the CSV in data/trondheim/ or set DATASET_PATH.')
    df = pd.DataFrame()
else:
    print('Loading', found)
    try:
        df = robust_read_ev(found)
        print('Loaded — rows:', len(df), 'columns:', list(df.columns))
    except Exception as e:
        print('Error reading file robustly:', type(e).__name__, e)
        df = pd.DataFrame()

# quick peek if loaded
if not df.empty:
    display(df.head())
    print('Column types:')
    print(df.dtypes)


Loading data/trondheim/Dataset_2_Hourly_EV_per_user.csv
Loaded — rows: 88156 columns: ['date_from', 'date_to', 'User_ID', 'session_ID', 'Synthetic_3_6kW', 'Synthetic_7_2kW', 'Flex_3_6kW', 'Flex_7_2kW']


,date_from,date_to,User_ID,session_ID,Synthetic_3_6kW,Synthetic_7_2kW,Flex_3_6kW,Flex_7_2kW
0,2018-12-21 10:00:00,2018-12-21 11:00:00,AdO3-4,1.0,0.30,0.30,NaN,0.060
1,2018-12-21 10:00:00,2018-12-21 11:00:00,AdO3-4,2.0,0.87,0.87,NaN,0.114
2,2018-12-21 11:00:00,2018-12-21 12:00:00,AdO3-4,3.0,1.62,3.24,NaN,NaN
3,2018-12-21 12:00:00,2018-12-21 13:00:00,AdO3-4,3.0,3.60,7.20,NaN,NaN
4,2018-12-21 13:00:00,2018-12-21 14:00:00,AdO3-4,3.0,3.60,7.20,NaN,NaN


Column types:
date_from          datetime64[ns]
date_to            datetime64[ns]
User_ID                    object
session_ID                float64
Synthetic_3_6kW           float64
Synthetic_7_2kW           float64
Flex_3_6kW                float64
Flex_7_2kW                float64
dtype: object


## 3) EDA: shape, missingness, basic time parsing and counts

In [6]:
def basic_eda(df, timestamp_cols_hint=None):
    if df.empty:
        print('DataFrame empty — load dataset first (see previous cell).')
        return

    print('Rows:', len(df))
    print('Columns:', len(df.columns))
    print('Missing values (top 20):')
    print(df.isna().sum().sort_values(ascending=False).head(20))

    # Try to find a timestamp column
    ts = None
    candidates = ['timestamp', 'start_time', 'datetime', 'date', 'time', 'usage_timestamp']
    if timestamp_cols_hint:
        candidates = list(timestamp_cols_hint) + candidates

    for c in candidates:
        if c in df.columns:
            ts = c
            break

    if ts is not None:
        print('Found timestamp column:', ts)
        try:
            df['_ts'] = pd.to_datetime(df[ts])
        except Exception:
            df['_ts'] = pd.to_datetime(df[ts], errors='coerce')

        print('Time span:', df['_ts'].min(), '->', df['_ts'].max())
        df['_date'] = df['_ts'].dt.date
        # events per day
        daily = df.groupby('_date').size()
        print('Days with data:', daily.shape[0])
        print('Daily events — median:', int(daily.median()), 'mean:', int(daily.mean()))
        plt.figure(figsize=(10,3))
        daily.plot(title='Events per day')
        plt.show()
    else:
        print('No obvious timestamp column found — inspect column names to find time info.')

    # Try to find an energy/charge column
    energy_candidates = ['energy_kwh', 'energy', 'charge_kwh', 'kwh', 'energy_kWh', 'Charging_kWh']
    energy_col = None
    for c in energy_candidates:
        if c in df.columns:
            energy_col = c
            break

    if energy_col is not None:
        print('Found energy column:', energy_col)
        print(df[energy_col].describe())
        plt.figure(figsize=(6,3))
        sns.histplot(df[energy_col].dropna(), bins=50, kde=False)
        plt.title('Energy distribution')
        plt.show()

    # If there is a user id column, show distribution per user
    user_candidates = ['user_id', 'user', 'meter_id', 'ev_user']
    uid = None
    for c in user_candidates:
        if c in df.columns:
            uid = c
            break

    if uid is not None:
        counts = df[uid].value_counts()
        print('Unique users:', counts.shape[0])
        print('Top users sample counts:')
        display(counts.head(10))
        plt.figure(figsize=(6,3))
        sns.histplot(counts.values, bins=50)
        plt.xlabel('Events per user')
        plt.show()

basic_eda(df)

Rows: 88156
Columns: 8
Missing values (top 20):
Synthetic_7_2kW    69199
Synthetic_3_6kW    57046
Flex_3_6kW         26070
Flex_7_2kW         15532
User_ID              390
session_ID           390
date_from              0
date_to                0
dtype: int64
No obvious timestamp column found — inspect column names to find time info.


## 4) Dataset sufficiency heuristics

In [26]:
def assess_sufficiency(df, problem='classification'):
    # Returns a verdict and diagnostic numbers. Heuristics are intentionally conservative and
    # intended as discussion points for the class.
    if df.empty:
        return {'verdict': 'no_data', 'message': 'No data available in DataFrame.'}

    n = len(df)
    users = None
    for c in ['user_id', 'user', 'meter_id', 'ev_user']:
        if c in df.columns:
            users = df[c].nunique()
            break

    # time span if available
    if '_ts' in df.columns and not df['_ts'].isna().all():
        span_days = (df['_ts'].max() - df['_ts'].min()).days
    else:
        span_days = None

    # simple rules of thumb (class discussion):
    # - For a small NN on tabular data, O(10k) rows is a reasonable minimum; for complex temporal models more is needed.
    # - If many users but few events per user, per-user modeling will be hard.

    verdict = 'uncertain'
    reasons = []

    if n < 1000:
        verdict = 'insufficient'
        reasons.append('Less than 1k total rows — too small for a neural net from scratch')
    elif n < 5000:
        verdict = 'borderline'
        reasons.append('Between 1k and 5k rows — may be possible for tiny models or heavy regularization')
    elif n < 20000:
        verdict = 'ok_for_small_nn'
        reasons.append('Between 5k and 20k rows — feasible for small MLPs with careful validation')
    else:
        verdict = 'good'
        reasons.append('More than 20k rows — generally enough data for training a modest NN')

    if users is not None:
        avg_per_user = n / float(users) if users>0 else None
        if avg_per_user is not None and avg_per_user < 5:
            reasons.append('Very few events per user — personalizing per-user models will be hard')

    return {'verdict': verdict, 'n_rows': n, 'n_users': users, 'span_days': span_days, 'reasons': reasons}

# Run assessment
assess = assess_sufficiency(df)
print(assess)

{'verdict': 'good', 'n_rows': 88156, 'n_users': None, 'span_days': None, 'reasons': ['More than 20k rows — generally enough data for training a modest NN']}


## 5) Baseline model (classification example)

In [ ]:
# Simplified classification baseline: use hourly `feats` only and avoid external helper functions
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

feat_path = Path('data/processed/ev_features.csv')
if not feat_path.exists():
    print('Features file not found:', feat_path, '— run the prep notebook first')
else:
    feats = pd.read_csv(feat_path, index_col=0, parse_dates=True)
    feats = feats.sort_index()
    # ensure we have a next-hour numeric target 'y'
    if 'y' not in feats.columns:
        if 'total_energy_kWh' in feats.columns:
            feats['y'] = feats['total_energy_kWh'].shift(-1)
        else:
            print('No numeric energy column found to create target; skipping classification baseline')
            feats = None

    if feats is None or 'y' not in feats.columns:
        print('Skipping baseline — no target available')
    else:
        feats2 = feats.dropna(subset=['y']).copy()
        # binary target: whether next hour has positive energy
        feats2['next_hour_charge'] = (feats2['y'] > 0).astype(int)

        # numeric features only; drop the numeric target columns if present
        X_df = feats2.select_dtypes(include=[np.number]).copy()
        for drop_col in ['y', 'total_energy_kWh', 'next_hour_charge']:
            if drop_col in X_df.columns:
                X_df = X_df.drop(columns=[drop_col])

        # fill numeric NaNs with 0 (user requested behavior)
        X_df = X_df.fillna(0)
        X = X_df.values
        y = feats2['next_hour_charge'].values

        # Guard: need at least two classes
        y_series = pd.Series(y)
        if y_series.nunique() < 2:
            print('Only one class present in the target; skipping classifier training (unique values:)', y_series.unique())
        else:
            # time-based split (no shuffle)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            clf = LogisticRegression(max_iter=200)
            clf.fit(X_train, np.asarray(y_train))
            preds = clf.predict(X_test)
            prob = clf.predict_proba(X_test)[:, 1] if hasattr(clf, 'predict_proba') else None
            print('Baseline logistic — accuracy:', accuracy_score(np.asarray(y_test), np.asarray(preds)))
            if prob is not None and len(np.unique(y_test)) > 1:
                try:
                    print('ROC AUC:', roc_auc_score(np.asarray(y_test), np.asarray(prob)))
                except Exception as e:
                    print('ROC AUC calculation error:', e)


Baseline logistic — accuracy: 0.9974398361495136
ROC AUC: 0.9998973305954826


In [ ]:
# Tiny PyTorch classifier run immediately after the baseline — uses X_train/X_test/y_train/y_test from the baseline
# This cell is intentionally self-contained so nbconvert executes it reliably.
import numpy as np
import math
from sklearn.metrics import accuracy_score

if 'X_train' in globals() and 'y_train' in globals() and 'X_test' in globals() and 'y_test' in globals():
    try:
        import torch
        import torch.nn as nn
        import torch.optim as optim
        import torch.utils.data as data_utils
        has_torch_local = True
    except Exception:
        has_torch_local = False

    if not has_torch_local:
        print('Torch not available in this kernel — skipping NN classifier run')
    else:
        # prepare
        X_tr = X_train.astype(np.float32)
        y_tr = np.asarray(y_train).astype(np.int64)
        X_te = X_test.astype(np.float32)
        y_te = np.asarray(y_test).astype(np.int64)

        train_ds = data_utils.TensorDataset(torch.from_numpy(X_tr), torch.from_numpy(y_tr))
        loader = data_utils.DataLoader(train_ds, batch_size=128, shuffle=True)

        n_features = X_tr.shape[1]
        n_classes = int(y_tr.max()) + 1 if len(np.unique(y_tr)) > 1 else 2

        class TinyClassifier(nn.Module):
            def __init__(self, in_dim, hid=64, out_dim=2):
                super().__init__()
                self.net = nn.Sequential(
                    nn.Linear(in_dim, hid),
                    nn.ReLU(),
                    nn.Linear(hid, hid),
                    nn.ReLU(),
                    nn.Linear(hid, out_dim)
                )
            def forward(self, x):
                return self.net(x)

        device_local = torch.device('cpu')
        try:
            if torch.backends.mps.is_available():
                device_local = torch.device('mps')
            elif torch.cuda.is_available():
                device_local = torch.device('cuda')
        except Exception:
            device_local = torch.device('cpu')

        model = TinyClassifier(n_features, hid=64, out_dim=n_classes).to(device_local)
        loss_fn = nn.CrossEntropyLoss()
        opt = optim.Adam(model.parameters(), lr=1e-3)

        epochs = 5
        for epoch in range(1, epochs+1):
            model.train()
            running = 0.0
            for xb, yb in loader:
                xb = xb.to(device_local)
                yb = yb.to(device_local)
                opt.zero_grad()
                logits = model(xb)
                loss = loss_fn(logits, yb)
                loss.backward()
                opt.step()
                running += float(loss.item())
            print(f'Epoch {epoch}/{epochs} — loss {running/len(loader):.4f}')

        # eval
        model.eval()
        with torch.no_grad():
            Xte_t = torch.from_numpy(X_te).to(device_local)
            logits = model(Xte_t)
            preds = logits.argmax(dim=1).cpu().numpy()
        try:
            print('NN test acc:', accuracy_score(y_te, preds))
        except Exception:
            print('NN test predictions computed (accuracy_score unavailable or failed)')
else:
    print('Required training variables (X_train/y_train/X_test/y_test) not found — ensure baseline cell ran before this cell')

## 6) Tiny PyTorch MLP prototype

In [ ]:
# Guarded Tiny PyTorch MLP prototype
# Only attempt to run if torch is available and training arrays are present
if globals().get('has_torch', False) and 'X_train' in globals() and X_train is not None and 'y_train' in globals() and y_train is not None:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    import torch.utils.data as data_utils

    # Convert to float32 / int64 as required
    X_small = X_train.astype(np.float32)
    y_small = y_train.astype(np.int64)

    train_tensor = data_utils.TensorDataset(torch.from_numpy(X_small), torch.from_numpy(y_small))
    loader = data_utils.DataLoader(train_tensor, batch_size=64, shuffle=True)

    n_features = X_small.shape[1] if X_small.ndim > 1 else 1
    n_classes = int(y_small.max()) + 1 if len(np.unique(y_small)) > 1 else 2

    class TinyMLP(nn.Module):
        def __init__(self, in_dim, hid=64, out_dim=2):
            super().__init__()
            self.net = nn.Sequential(
                nn.Linear(in_dim, hid),
                nn.ReLU(),
                nn.Linear(hid, hid),
                nn.ReLU(),
                nn.Linear(hid, out_dim)
            )
        def forward(self, x):
            return self.net(x)

    device_local = device if device is not None else torch.device('cpu')
    model = TinyMLP(n_features, hid=64, out_dim=n_classes).to(device_local)
    loss_fn = nn.CrossEntropyLoss()
    opt = optim.Adam(model.parameters(), lr=1e-3)

    epochs = 5
    for epoch in range(1, epochs+1):
        model.train()
        running = 0.0
        for xb, yb in loader:
            xb = xb.to(device_local)
            yb = yb.to(device_local)
            opt.zero_grad()
            logits = model(xb)
            loss = loss_fn(logits, yb)
            loss.backward()
            opt.step()
            running += float(loss.item())
        print(f'Epoch {epoch}/{epochs} — loss {running/len(loader):.4f}')

    # quick eval on test set if available
    model.eval()
    if 'X_test' in globals() and X_test is not None and 'y_test' in globals() and y_test is not None:
        with torch.no_grad():
            X_test_t = torch.from_numpy(X_test.astype(np.float32)).to(device_local)
            logits = model(X_test_t)
            preds = logits.argmax(dim=1).cpu().numpy()
            try:
                print('NN test acc:', accuracy_score(y_test, preds))
            except Exception:
                print('NN test predictions computed (accuracy_score unavailable or failed)')
else:
    print('Skipping PyTorch prototype (torch not available, or training data not prepared)')


Epoch 1/5 — loss 0.2608
Epoch 2/5 — loss 0.1532
Epoch 3/5 — loss 0.1310
Epoch 4/5 — loss 0.1202
Epoch 5/5 — loss 0.1114
NN test acc: 0.9974398361495136


In [ ]:
# Regression MLP (self-contained) — trains a tiny PyTorch regressor on features and reports MAE/RMSE
import numpy as np
import math
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

feat_path = Path('data/processed/ev_features.csv')
if not feat_path.exists():
    print('Features file not found at', feat_path, '-- skipping regression MLP in main notebook')
else:
    feats = pd.read_csv(feat_path, index_col=0, parse_dates=True)
    feats = feats.sort_index()
    if 'y' not in feats.columns:
        if 'total_energy_kWh' in feats.columns:
            feats['y'] = feats['total_energy_kWh'].shift(-1)
        else:
            print('No numeric energy column found to create target; skipping regression MLP')
            feats = None
    if feats is not None:
        feats2 = feats.dropna(subset=['y']).copy()
        X_df = feats2.select_dtypes(include=[np.number]).copy()
        for drop_col in ['y', 'total_energy_kWh']:
            if drop_col in X_df.columns:
                X_df = X_df.drop(columns=[drop_col])
        X_df = X_df.fillna(0)
        const_cols = [c for c in X_df.columns if X_df[c].nunique(dropna=False) <= 1]
        if const_cols:
            X_df = X_df.drop(columns=const_cols)
        X = X_df.values.astype(np.float32)
        y = feats2['y'].values.astype(np.float32)

        # time-based split
        split = int(len(X) * 0.8)
        X_train_r, X_test_r = X[:split], X[split:]
        y_train_r, y_test_r = y[:split], y[split:]
        scaler = StandardScaler()
        X_train_r = scaler.fit_transform(X_train_r)
        X_test_r = scaler.transform(X_test_r)

        # Try PyTorch
        try:
            import torch
            import torch.nn as nn
            import torch.optim as optim
            has_torch_local = True
        except Exception:
            has_torch_local = False

        if has_torch_local:
            class TinyReg(nn.Module):
                def __init__(self, in_dim, hid=64):
                    super().__init__()
                    self.net = nn.Sequential(nn.Linear(in_dim, hid), nn.ReLU(), nn.Linear(hid, hid), nn.ReLU(), nn.Linear(hid, 1))
                def forward(self, x):
                    return self.net(x).squeeze(-1)

            device_local = torch.device('cpu')
            try:
                if torch.backends.mps.is_available():
                    device_local = torch.device('mps')
                elif torch.cuda.is_available():
                    device_local = torch.device('cuda')
            except Exception:
                device_local = torch.device('cpu')

            model = TinyReg(X_train_r.shape[1], hid=64).to(device_local)
            loss_fn = nn.MSELoss()
            opt = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
            epochs = 16
            batch_size = 256
            Xt = torch.from_numpy(X_train_r).to(device_local)
            yt = torch.from_numpy(y_train_r).to(device_local)
            dataset = torch.utils.data.TensorDataset(Xt, yt)
            loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
            for epoch in range(1, epochs+1):
                model.train()
                running = 0.0
                for xb, yb in loader:
                    xb = xb.to(device_local)
                    yb = yb.to(device_local)
                    opt.zero_grad()
                    preds = model(xb)
                    loss = loss_fn(preds, yb)
                    loss.backward()
                    opt.step()
                    running += float(loss.item())
                print(f'Reg Epoch {epoch}/{epochs} — train loss: {running/len(loader):.6f}')
            model.eval()
            with torch.no_grad():
                Xte = torch.from_numpy(X_test_r).to(device_local)
                preds = model(Xte).cpu().numpy()
            mae = mean_absolute_error(y_test_r, preds)
            rmse = math.sqrt(mean_squared_error(y_test_r, preds))
            print('Regression TinyReg — MAE:', round(mae,6), 'RMSE:', round(rmse,6))
        else:
            print('Torch not available — falling back to LinearRegression for regression test')
            lr = LinearRegression()
            lr.fit(X_train_r, y_train_r)
            preds = lr.predict(X_test_r)
            mae = mean_absolute_error(y_test_r, preds)
            rmse = math.sqrt(mean_squared_error(y_test_r, preds))
            print('Regression LinearReg — MAE:', round(mae,6), 'RMSE:', round(rmse,6))

## 7) Summary and next steps

- The notebook computed simple heuristics (rows, users, time span) and ran a lightweight baseline and a tiny NN when possible.
- If the dataset has >20k rows and reasonable per-user events, training a small NN from scratch is usually feasible.
- If the dataset is smaller (<5k), prefer feature-engineered classical models, transfer learning, per-user aggregation, or data augmentation/synthetic generation.

Next steps for the class project:
1. Confirm dataset CSV availability and column names.
2. Decide the prediction task (next-hour charging probability vs. energy amount).
3. Create a richer feature set (time-of-day, weekday, rolling counts, weather merge).
4. Train stronger temporal models (RNN/Transformer) only if data span and volume justify it.

---
If you want, I can now: load the actual CSV from your workspace and run the notebook cells here (if you want me to execute code), or iterate the notebook to add more focused feature engineering (e.g., time features, merge with weather).